In [1]:
# Import 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [7]:
from sklearn.datasets import load_boston
housing = load_boston()
housing
house = pd.DataFrame(housing['data'])
house.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [9]:
house[13] = housing['target']
house.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [97]:
X = house.drop(13,axis=1)
y = house[13]
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [65]:
### RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
rf_clf = RandomForestRegressor()

In [66]:
rf_clf.fit(X_train,y_train)

RandomForestRegressor()

In [67]:
rf_clf.score(X_test,y_test)

0.8796040809716893

In [64]:
## Predicit
y_pred = rf_clf.predict(X_test)
y_pred[:5]

array([11.25 , 24.334, 12.373, 22.112, 31.301])

In [18]:
y_test[:5]

412    17.9
326    23.0
403     8.3
265    22.8
275    32.0
Name: 13, dtype: float64

In [19]:
from sklearn.metrics import mean_absolute_error
MAE = mean_absolute_error(y_test,y_pred)

In [20]:
MAE  # $ 2.3 $  # MAE = 0 is the best 

2.330892156862746

In [22]:
# MAE on training data
y_pred_train = rf_clf.predict(X_train)

In [23]:
MAE_T = mean_absolute_error(y_train,y_pred_train)

In [24]:
MAE_T

0.8263638613861372

In [25]:
# Mean square error // Amplified Error
from sklearn.metrics import mean_squared_error
MSE = mean_squared_error(y_test,y_pred)

In [26]:
MSE

11.07781581372549

In [27]:
MSE_T= mean_squared_error(y_train,y_pred_train)

In [28]:
MSE_T

1.5009147351485137

In [29]:
# Imporove Model
# 1. By Hand
rf_clf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [31]:
# Split data int train,validate,test
house.shape

(506, 14)

In [40]:
l1 = int(np.ceil((house.shape[0]/100)*80))
l2 = int(np.ceil((house.shape[0]/100)*90))

In [46]:
# 506 => 80% => 506
X_Train = X[:l1]
X_Valide = X[l1:l2]
X_Test = X[l2:]

y_Train = y[:l1]
y_Valide = y[l1:l2]
y_Test = y[l2:]

In [45]:
print(X_Train.shape)
print(X_Valide.shape)
print(X_Test.shape)

(405, 13)
(51, 13)
(50, 13)


In [78]:
# Manual Adjust
rf_clf_2 = RandomForestRegressor(n_estimators=200)
rf_clf_2.fit(X_Train,y_Train)

RandomForestRegressor(n_estimators=200)

In [79]:
rf_clf_2.score(X_Valide,y_Valide)

-0.16403562814091877

In [92]:
# RandomSearchCV h-parameter adjust
rf = RandomForestRegressor()
from sklearn.model_selection import RandomizedSearchCV

grid = {
    'n_estimators':[50,100,150,200,250,300], # 6
    'criterion':['mse','mae'], # 2
    'max_features':['auto','sqrt','log2'], # 3 => 6x2x3 = 36
}

rand_clf = RandomizedSearchCV(
        rf,
        grid,
        n_iter=10,
        verbose = 2,
        cv = 5
)

rand_clf.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END ..criterion=mae, max_features=auto, n_estimators=50; total time=   2.8s
[CV] END ..criterion=mae, max_features=auto, n_estimators=50; total time=   1.7s
[CV] END ..criterion=mae, max_features=auto, n_estimators=50; total time=   1.3s
[CV] END ..criterion=mae, max_features=auto, n_estimators=50; total time=   1.4s
[CV] END ..criterion=mae, max_features=auto, n_estimators=50; total time=   1.3s
[CV] END ..criterion=mse, max_features=sqrt, n_estimators=50; total time=   0.4s
[CV] END ..criterion=mse, max_features=sqrt, n_estimators=50; total time=   0.1s
[CV] END ..criterion=mse, max_features=sqrt, n_estimators=50; total time=   0.0s
[CV] END ..criterion=mse, max_features=sqrt, n_estimators=50; total time=   0.1s
[CV] END ..criterion=mse, max_features=sqrt, n_estimators=50; total time=   0.2s
[CV] END .criterion=mse, max_features=log2, n_estimators=100; total time=   0.3s
[CV] END .criterion=mse, max_features=log2, n_es

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(),
                   param_distributions={'criterion': ['mse', 'mae'],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'n_estimators': [50, 100, 150, 200, 250,
                                                         300]},
                   verbose=2)

In [93]:
rand_clf.best_params_

{'n_estimators': 100, 'max_features': 'sqrt', 'criterion': 'mae'}

In [94]:
rand_clf.score(X_test,y_test)

0.8753235841669673

In [99]:
predict = rand_clf.predict(X_test)
mean_absolute_error(y_test,predict)

1.1277107843137246

In [101]:
mean_squared_error(y_test,predict)

2.5203032328431347